## Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [76]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_dropped = application_df.drop(columns=['EIN', 'NAME', 'USE_CASE','SPECIAL_CONSIDERATIONS'])

In [79]:
# Determine the number of unique values in each column.
unique_values = application_df_dropped.nunique()
unique_values

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [80]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df_dropped['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [157]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_counts[application_type_counts < 200].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_dropped['APPLICATION_TYPE'] = application_df_dropped['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df_dropped['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [158]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df_dropped['CLASSIFICATION'].value_counts()

classification_counts

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [159]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [160]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts < 900].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df_dropped['CLASSIFICATION'] = application_df_dropped['CLASSIFICATION'].replace(cls, "Other")


# Check to make sure replacement was successful
application_df_dropped['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [161]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df_dropped)

In [162]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(columns=['IS_SUCCESSFUL'])
y = application_df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [163]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [164]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 80)                2720      
                                                                 
 dense_37 (Dense)            (None, 30)                2430      
                                                                 
 dense_38 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,181
Trainable params: 5,181
Non-trainable params: 0
_________________________________________________________________


In [165]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [166]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
858/858 [==============================] - 2s 1ms/step - loss: 0.5771 - accuracy: 0.7138
Epoch 2/10
858/858 [==============================] - 1s 1ms/step - loss: 0.5615 - accuracy: 0.7249
Epoch 3/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5586 - accuracy: 0.7255
Epoch 4/10
858/858 [==============================] - 1s 1ms/step - loss: 0.5576 - accuracy: 0.7258
Epoch 5/10
858/858 [==============================] - 1s 1ms/step - loss: 0.5576 - accuracy: 0.7256
Epoch 6/10
858/858 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7267
Epoch 7/10
858/858 [==============================] - 1s 1ms/step - loss: 0.5559 - accuracy: 0.7281
Epoch 8/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5550 - accuracy: 0.7271
Epoch 9/10
858/858 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7284
Epoch 10/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5545 - accuracy: 0.7274

In [167]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5668 - accuracy: 0.7203 - 358ms/epoch - 2ms/step
Loss: 0.5667912364006042, Accuracy: 0.7202624082565308


In [38]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")